# Test gurobipy

based on Zhenping Guo's Homework

In [1]:
import gurobipy as gp
from gurobipy import GRB 
import sys

In [2]:
# load data
week, demand, minInventory, holdCostUnit = gp.multidict({
    1 : [10, 1, 1],
    2 : [10, 1, 1],
    3 : [10, 1, 1],
    4 : [0, 0, 1],
    5 : [0, 0, 1],
    6 : [15, 1.5,1],
    7 : [20, 2, 1],
    8 : [20, 2, 1],
    9 : [0, 0, 1],
    10 :[10, 1, 1]})

In [3]:
gp.multidict({
    1 : [10, 1, 1],
    2 : [10, 1, 1],
    3 : [10, 1, 1],
    4 : [0, 0, 1],
    5 : [0, 0, 1],
    6 : [15, 1.5,1],
    7 : [20, 2, 1],
    8 : [20, 2, 1],
    9 : [0, 0, 1],
    10 :[10, 1, 1]})

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 {1: 10, 2: 10, 3: 10, 4: 0, 5: 0, 6: 15, 7: 20, 8: 20, 9: 0, 10: 10},
 {1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 1.5, 7: 2, 8: 2, 9: 0, 10: 1},
 {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}]

In [4]:
week

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
demand

{1: 10, 2: 10, 3: 10, 4: 0, 5: 0, 6: 15, 7: 20, 8: 20, 9: 0, 10: 10}

In [6]:
holdCostUnit

{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}

In [7]:
M_ = sum(demand[idx] for idx in demand)
M_

95

In [8]:
# build Gurobi Model
m = gp.Model("IE522_zguo19_proj3")

# Create decision variables
x = m.addVars(week,lb= 0,  name="x")  #order amount each week
y = m.addVars(week,lb= 0,  name="y")  #Inventory held at end of week
z = m.addVars(week,4,vtype=GRB.BINARY, name="z")   #auxiliary variable

M = sum(demand)
p = {i:2*z[i,1]+z[i,2]+0.5*z[i,3] for i in week} #price in unit

# obj: minimize costs
m.setObjective(x.prod(p)+y.prod(holdCostUnit), GRB.MINIMIZE)

# Add constraints
m.addConstrs(y[i]>=minInventory[i] for i in week)
m.addConstr(y[1]==20+x[1]-demand[1])
m.addConstrs(y[i+1]==y[i]+x[i+1]-demand[i+1] for i in range(1,len(week)))
m.addConstrs(z.sum(i,'*')==1 for i in week)
m.addConstrs(x[i]<=9.99*z[i,1]+14.99*z[i,2]+M*z[i,3] for i in week)
m.addConstrs(x[i]>=5*z[i,1]+10*z[i,2]+15*z[i,3] for i in week)

Restricted license - for non-production use only - expires 2023-10-25


{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [9]:
z.sum(1,'*') 

<gurobi.LinExpr: <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*>>

In [10]:
p

{1: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 2: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 3: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 4: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 5: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 6: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*>>,
 7: <gurobi.LinExpr: 2.0 <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awai

In [11]:
x.prod(p)

<gurobi.QuadExpr: 0.0 + [ 2.0 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 2.0 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 2.0 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 2.0 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 0.5 <gurobi.Var *Awaiting Model Update*> * <gurobi.Var *Awaiting Model Update*> + 2.0 <g

In [12]:
z

{(1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 0): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>,
 (5, 0): <gurobi.Var *Awaiting Model Update*>,
 (5, 1): <gurobi.Var *Awaiting Model Update*>,
 (5, 2): <gurobi.Var *Awaiting Model Update*>,
 (5, 3): <gurobi.Var *Awaiting Model Update*>,
 (6, 0): <gurobi.Var *Awaiting Model Update*>,
 (6, 1): <gur

In [13]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 50 rows, 60 columns and 159 nonzeros
Model fingerprint: 0xb68c33a9
Model has 30 quadratic objective terms
Variable types: 20 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  QObjective range [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 159.9450000
Presolve removed 14 rows and 15 columns
Presolve time: 0.01s
Presolved: 66 rows, 75 columns, 222 nonzeros
Variable types: 45 continuous, 30 integer (30 binary)

Root relaxation: objective 2.719897e+01, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.19897    0    7  159.9

In [14]:
GRB.UNBOUNDED

5

In [15]:
GRB.OPTIMAL

2

In [16]:
m.Status

2

In [17]:
status=m.status

if status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
    sys.exit()
if status == GRB.OPTIMAL:
    print('\nCost: %g' % m.ObjVal)
    print('\norder:')
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))  
    sys.exit()
if status !=GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print('Optimization was stopped with status %d'  % status)
    sys.exit()

# do IIS
print('The model is infeasible; computing IIS')
m.computeIIS()
if m.IISMinimal:
    print('IIS is minimal\n')
else:
    print('IIS is not minimal\n')
print('\nThe following constraint(s) cannot be satisfied:')
for c in m.getConstrs():
    if c.IISConstr:
        print('%s' % c.ConstrName)


Cost: 82.5

order:
x[1] 0
x[2] 11
x[3] 0
x[4] 0
x[5] 0
x[6] 15.5
x[7] 20.5
x[8] 20
x[9] 0
x[10] 10
y[1] 10
y[2] 11
y[3] 1
y[4] 1
y[5] 1
y[6] 1.5
y[7] 2
y[8] 2
y[9] 2
y[10] 2
z[1,0] 1
z[1,1] 0
z[1,2] -0
z[1,3] -0
z[2,0] 0
z[2,1] 0
z[2,2] 1
z[2,3] 0
z[3,0] 1
z[3,1] 0
z[3,2] 0
z[3,3] 0
z[4,0] 1
z[4,1] -0
z[4,2] -0
z[4,3] -0
z[5,0] 1
z[5,1] -0
z[5,2] -0
z[5,3] -0
z[6,0] 0
z[6,1] 0
z[6,2] 0
z[6,3] 1
z[7,0] 0
z[7,1] -0
z[7,2] 0
z[7,3] 1
z[8,0] 0
z[8,1] -0
z[8,2] 0
z[8,3] 1
z[9,0] 1
z[9,1] 0
z[9,2] 0
z[9,3] -0
z[10,0] 0
z[10,1] 0
z[10,2] 1
z[10,3] -0


SystemExit: 

/Users/buxinshe/mambaforge/envs/andes/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
